In [19]:
from openai import OpenAI
openai_client = OpenAI()  # Assumes OPENAI_API_KEY is set in environment

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [3]:
from minsearch import AppendableIndex

## Setup and Prepare Data

In [5]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [6]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
index = AppendableIndex(
    text_fields=["question", "text", "section"]
)

index.fit(documents)

## Agentic RAG

The biggest difference between traditional RAG (fixed process + rigid) and Agentic RAG is that we let the LLM decide if it needs to invoke the tool or not. We achieve this by using function calling capabilities.

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
    )

    return results

In [12]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}


### Library Setup

In [10]:
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.chat.runners import DisplayingRunnerCallback
from toyaikit.tools import Tools


### Processing Function Calls



In [13]:
agent_tools = Tools()
agent_tools.add_tool(search, search_tool)


In [9]:
instructions = """
You're a course teaching assistant.
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call
""".strip()

question = "I just discovered the course. Can I still join it?"


LLMs are STATELESS, so we need to include conversation history

send back entire conversation history

In [17]:
chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=openai_client
)


#### Running the Agent

In [18]:
callback = DisplayingRunnerCallback(chat_interface)

question = 'how do I install kafka'
loop_result = runner.loop(prompt=question, callback=callback)


AttributeError: 'OpenAI' object has no attribute 'send_request'

Now we are ready to send the results back to the model.

invoke API with function call and function call results w/ previous conversation history

In [ ]:
response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)

In [ ]:
response.output_text

### Adding Explanations

make agent explain its decision-making process:

### Test

In [ ]:
tools = [search_tool]


chat_messages = [
    {"role": "developer", "content": instrunctions},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='gpt-40-mini',
    input=chat_messages,
    tools=tools
)

In [ ]:
response.output[0].content[0].text

The second part contains the function call details.

Making these calls manually be executing the notebook cells is not convenient so let's write some code for automating it.

## Agentic Loop

create more flexible function for handling different types of calls

now the loop